In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [9]:
# reflect an existing database into a new model
Base = automap_base()


['measurement', 'station']

In [10]:
# reflect the tables
Base.prepare(autoload_with=engine)

C:\Users\griff\AppData\Local\Temp\ipykernel_20512\2394540615.py:2: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.measurement, and will be replaced in the string-lookup table.
  Base.prepare(autoload_with=engine)
C:\Users\griff\AppData\Local\Temp\ipykernel_20512\2394540615.py:2: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.station, and will be replaced in the string-lookup table.
  Base.prepare(autoload_with=engine)


In [11]:
# View all of the classes that automap found
Base.classes.keys() 

['measurement', 'station']

In [ ]:
# Save references to each table


In [8]:
# Create our session (link) from Python to the DB
